# Classical Extensional Logic in Logos Theory

This notebook demonstrates the fundamental operators of classical propositional logic within the Logos semantic framework. Extensional logic deals with truth-functional operators where the truth value of complex propositions depends solely on the truth values of their components.

We'll explore both **invalid arguments** (countermodels) and **valid arguments** (theorems) to understand the boundaries and capabilities of classical logic.

## Setup and Theory Configuration

In [1]:
# Import required modules for model checking
import sys
from model_checker.jupyter import create_build_example, build_and_check
from model_checker.theory_lib import logos

# Load extensional operators - classical logic only
from model_checker.theory_lib.logos.semantic import LogosSemantics, LogosProposition, LogosModelStructure
from model_checker.theory_lib.logos.operators import LogosOperatorRegistry

# Create operator registry for extensional theory
ext_registry = LogosOperatorRegistry()
ext_registry.load_subtheories(['extensional'])

# Define the semantic theory with classical operators
ext_theory = {
    "semantics": LogosSemantics,
    "proposition": LogosProposition,
    "model": LogosModelStructure,
    "operators": ext_registry.get_operators(),
}

print(f"Loaded extensional theory with {len(ext_theory['operators'].operator_dictionary)} classical operators")
print(f"Available operators: {', '.join(ext_theory['operators'].operator_dictionary.keys())}")

Loaded extensional theory with 7 classical operators
Available operators: \neg, \wedge, \vee, \top, \bot, \rightarrow, \leftrightarrow


---

## Example 1: The Fallacy of Affirming the Consequent

### Background
One of the most common logical fallacies is *affirming the consequent*. This occurs when we observe that B is true, know that "if A then B", and incorrectly conclude that A must be true. This reasoning is invalid because B could be true for reasons other than A.

### The Argument
- **Premises**: B is true, A → B (if A then B)
- **Invalid Conclusion**: A is true
- **Why it fails**: B being true doesn't tell us which antecedent caused it

In [ ]:
# EXT_CM_2: Test affirming the consequent fallacy
EXT_CM_2_example = [
    ['B', '(A \\rightarrow B)'],  # Premises: B is true, and if A then B
    ['A'],                         # Conclusion: A is true (invalid!)
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 1,
        'iterate': 3,
        'expectation': True,  # We expect a countermodel (invalid argument)
    }
]

print("Running model checker...")
model = create_build_example('EXT_CM_2', ext_theory, EXT_CM_2_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'EXT_CM_2',
        'Affirming the Consequent',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected - this should be invalid!)")

### Result Interpretation
The countermodel shows a situation where both premises are true (B is true and A → B holds) but the conclusion A is false. This demonstrates that the argument form is invalid. In the model, B can be true independently of A, which is precisely what makes affirming the consequent a fallacy.

---

## Example 2: Modus Ponens (Valid Inference)

### Background
*Modus Ponens* is one of the most fundamental valid inference rules in logic. It states that if we know "if A then B" and we know that A is true, we can validly conclude that B is true. This form of reasoning is the backbone of mathematical proofs and logical deduction.

### The Argument
- **Premises**: A is true, A → B (if A then B)
- **Valid Conclusion**: B is true
- **Why it works**: The conditional guarantees B whenever A holds

In [ ]:
# EXT_TH_1: Test Modus Ponens - fundamental valid inference
EXT_TH_1_example = [
    ['A', '(A \\rightarrow B)'],  # Premises: A is true, and if A then B
    ['B'],                         # Conclusion: B is true (valid!)
    {
        'N': 3,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 1,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid argument)
    }
]

print("Running model checker...")
model = create_build_example('EXT_TH_1', ext_theory, EXT_TH_1_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to Modus Ponens!")
    model.model_structure.print_to(
        model.settings,
        'EXT_TH_1',
        'Modus Ponens',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Modus Ponens")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nIn every model where A is true and A → B holds, B must be true.")
    print("=" * 70)

### Result Interpretation
No countermodel can be found because Modus Ponens is a valid argument form. The semantic framework confirms that in every possible model where both premises are satisfied, the conclusion necessarily follows. This validates our logical intuition about this fundamental inference rule.

---

## Example 3: De Morgan's Law (First Form)

### Background
*De Morgan's Laws* are fundamental equivalences in logic that relate conjunction and disjunction through negation. The first law states that "not (A and B)" is equivalent to "(not A) or (not B)". This principle is crucial in simplifying logical expressions and in digital circuit design.

### The Argument
- **Premise**: ¬(A ∧ B) (not both A and B)
- **Valid Conclusion**: (¬A ∨ ¬B) (either not A or not B)
- **Why it works**: If it's not the case that both hold, at least one must fail

In [ ]:
# EXT_TH_7: Test De Morgan's Law (First Form)
EXT_TH_7_example = [
    ['\\neg (A \\wedge B)'],         # Premise: Not (A and B)
    ['(\\neg A \\vee \\neg B)'],     # Conclusion: (Not A) or (Not B)
    {
        'N': 3,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 1,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (valid law)
    }
]

print("Running model checker...")
model = create_build_example('EXT_TH_7', ext_theory, EXT_TH_7_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to De Morgan's Law!")
    model.model_structure.print_to(
        model.settings,
        'EXT_TH_7',
        'De Morgan\'s First Law',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: De Morgan's First Law")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThe negation of a conjunction equals the disjunction of negations.")
    print("=" * 70)

### Result Interpretation
The absence of a countermodel confirms De Morgan's First Law as a logical theorem. This law is not just a convenient equivalence but a necessary truth of classical logic, holding in all possible models.

---

## Example 4: Law of Excluded Middle

### Background
The *Law of Excluded Middle* states that for any proposition A, either A or not-A must be true. This is a cornerstone of classical logic, though it's rejected in some non-classical logics like intuitionistic logic. It expresses the idea that every proposition has a definite truth value.

### The Argument
- **Premises**: None (this is a logical law)
- **Valid Conclusion**: A ∨ ¬A (A or not A)
- **Why it works**: In classical logic, every proposition is either true or false

In [ ]:
# EXT_TH_6: Test Law of Excluded Middle
EXT_TH_6_example = [
    [],                            # No premises - this is a tautology
    ['(A \\vee \\neg A)'],         # Conclusion: A or not A
    {
        'N': 3,
        'contingent': False,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 1,
        'iterate': 1,
        'expectation': False,  # We expect no countermodel (tautology)
    }
]

print("Running model checker...")
model = create_build_example('EXT_TH_6', ext_theory, EXT_TH_6_example)

# Check validity
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to the Law of Excluded Middle!")
    model.model_structure.print_to(
        model.settings,
        'EXT_TH_6',
        'Law of Excluded Middle',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Law of Excluded Middle")
    print("=" * 70)
    print("No countermodel found - the tautology is VALID")
    print("\nThis tautology holds in every classical model without exception.")
    print("=" * 70)

### Result Interpretation
The Law of Excluded Middle is validated as a tautology - true in all models regardless of how we interpret the atomic propositions. This fundamental principle distinguishes classical logic from many non-classical alternatives.

---

## Summary

This notebook has demonstrated key aspects of classical extensional logic:

### Invalid Arguments (Countermodels)
- **Affirming the Consequent**: From B and A → B, cannot conclude A

### Valid Arguments (Theorems)
- **Modus Ponens**: From A and A → B, can conclude B
- **De Morgan's First Law**: ¬(A ∧ B) entails (¬A ∨ ¬B)
- **Law of Excluded Middle**: A ∨ ¬A is always true

These examples showcase the model-checking approach to logic: arguments are valid when no countermodel exists, and invalid arguments are demonstrated through explicit countermodels. The extensional operators (∧, ∨, ¬, →, ↔) form the foundation for more complex logical systems explored in other notebooks.